In [ ]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from keras.preprocessing.image import ImageDataGenerator


#Define input image dimensions
#Large images take too much time and resources.

In [ ]:
img_rows = 128
img_cols = 128
channels = 3
img_shape = (img_rows, img_cols, channels)

In [ ]:
!unzip Hydra.zip

In [ ]:
# !unzip /content/Common-Myna.zip

In [ ]:
# Create an ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(
    rotation_range=20,  # Random rotation up to 20 degrees
    horizontal_flip=True,  # Random horizontal flip
    vertical_flip=True,  # Random vertical flip
    width_shift_range=0.1,  # Random horizontal shift
    height_shift_range=0.1,  # Random vertical shift
)


In [ ]:
directory_path='/content/Hydra'
# directory_path='/content/Common-Myna'

file_paths=[]
image_arrays=[]
for filename in os.listdir(directory_path):
  file_path = os.path.join(directory_path,filename)
  file_paths.append(file_path)
for path in file_paths:
  image= Image.open(path)

  image_resized = image.resize((img_rows, img_cols))
  image_array= np.array(image_resized)
  if image_array.shape!= 3:
    image=image.convert('RGB')
    image_resized = image.resize((img_rows, img_cols))
    image_array= np.array(image_resized)
  # Apply data augmentation
    augmented_images = data_generator.flow(np.expand_dims(image_array, axis=0), batch_size=1)
    augmented_image = next(augmented_images)[0]

  print(path,' : ',image_array.shape)
  image_arrays.append(image_array)
  image_arrays.append(augmented_image)

image_dataset= np.array(image_arrays, dtype=np.float32)


/content/Hydra/Image_55.jpg  :  (128, 128, 3)
/content/Hydra/Image_56.jpg  :  (128, 128, 3)
/content/Hydra/Image_116.jpg  :  (128, 128, 3)
/content/Hydra/Image_28.jpg  :  (128, 128, 3)
/content/Hydra/Image_52.jpg  :  (128, 128, 3)
/content/Hydra/Image_89.jpg  :  (128, 128, 3)
/content/Hydra/Image_136.jpg  :  (128, 128, 3)
/content/Hydra/Image_35.jpg  :  (128, 128, 3)
/content/Hydra/Image_30.jpg  :  (128, 128, 3)
/content/Hydra/Image_46.JPG  :  (128, 128, 3)
/content/Hydra/Image_96.jpg  :  (128, 128, 3)
/content/Hydra/Image_58.jpg  :  (128, 128, 3)
/content/Hydra/Image_27.jpg  :  (128, 128, 3)
/content/Hydra/Image_40.jpg  :  (128, 128, 3)
/content/Hydra/Image_113.jpg  :  (128, 128, 3)
/content/Hydra/Image_111.jpg  :  (128, 128, 3)
/content/Hydra/Image_54.jpg  :  (128, 128, 3)
/content/Hydra/Image_130.jpg  :  (128, 128, 3)
/content/Hydra/Image_3.jpg  :  (128, 128, 3)
/content/Hydra/Image_128.jpg  :  (128, 128, 3)
/content/Hydra/Image_10.jpg  :  (128, 128, 3)
/content/Hydra/Image_51.jpg  

In [ ]:
for i in image_dataset:
  if i.shape[-1]!= 3:
    print(i.shape)




In [ ]:
# Xtrain=image_dataset
# print(Xtrain.shape)
# t=mnist.load_data()[0][0]
# print(t.shape)

In [ ]:
def build_generator():

    noise_shape = (100,) #1D array of size 100 (latent vector / noise)

#Define your generator network
#Here we are only using Dense layers. But network can be complicated based
#on the application. For example, you can use VGG for super res. GAN.

    model = Sequential()

    model.add(Dense(256, input_dim=100))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)    #Generated image

    return Model(noise, img)

#Alpha — α is a hyperparameter which controls the underlying value to which the
#function saturates negatives network inputs.
#Momentum — Speed up the training

In [ ]:
def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)
#The validity is the Discriminator’s guess of input being real or not.

#Now that we have constructed our two models it’s time to pit them against each other.
#We do this by defining a training function, loading the data set, re-scaling our training
#images and setting the ground truths.

In [ ]:
def train(epochs, batch_size=16, save_interval=50):

    # Load the dataset
    X_train= image_dataset

    # Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5

#Add channels dimension. As the input to our gen and discr. has a shape 28x28x1.
    # X_train = np.expand_dims(X_train, axis=3)
    half_batch = int(batch_size / 2)


#We then loop through a number of epochs to train our Discriminator by first selecting
#a random batch of images from our true dataset, generating a set of images from our
#Generator, feeding both set of images into our Discriminator, and finally setting the
#loss parameters for both the real and fake images, as well as the combined loss.

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]


        noise = np.random.normal(0, 1, (half_batch, 100))

        # Generate a half batch of fake images
        gen_imgs = generator.predict(noise)

        # Train the discriminator on real and fake images, separately
        #Research showed that separate training is more effective.
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    #take average loss from real and fake images.
    #
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

#And within the same loop we train our Generator, by setting the input noise and
#ultimately training the Generator to have the Discriminator label its samples as valid
#by specifying the gradient loss.
        # ---------------------
        #  Train Generator
        # ---------------------
#Create noise vectors as input for generator.
#Create as many noise vectors as defined by the batch size.
#Based on normal distribution. Output will be of size (batch size, 100)
        noise = np.random.normal(0, 1, (batch_size, 100))

        # The generator wants the discriminator to label the generated samples
        # as valid (ones)
        #This is where the genrator is trying to trick discriminator into believing
        #the generated image is true (hence value of 1 for y)
        valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size

        # Generator is part of combined where it got directly linked with the discriminator
        # Train the generator with noise as x and 1 as y.
        # Again, 1 as the output as it is adversarial and if generator did a great
        #job of folling the discriminator then the output would be 1 (true)
        g_loss = combined.train_on_batch(noise, valid_y)


#Additionally, in order for us to keep track of our training process, we print the
#progress and save the sample image output depending on the epoch interval specified.
# Plot the progress

        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if (epoch % save_interval == 0)or (d_loss[1]<0.6 and epoch > 10000):
            save_imgs(epoch)


In [ ]:
import os
def save_imgs(epoch):
    r, c = 4, 4
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5
    # Create the 'images/' directory if it doesn't exist
    os.makedirs('images_do/', exist_ok=True)
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            # axs[i,j].imshow(gen_imgs[cnt, :,:,0], )
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images_do/mnist_%d.png" % epoch)
    plt.close()
#This function saves our images for us to view

In [ ]:
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.

In [ ]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 49152)             0         
                                                                 
 dense_7 (Dense)             (None, 512)               25166336  
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 257       
                                                                 
Total params: 25,297,921
Trainable params: 25,297,921


In [ ]:
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 256)               25856     
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_11 (Dense)            (None, 512)               131584    
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_4 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [ ]:
z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)

In [ ]:
discriminator.trainable = False

In [ ]:
valid = discriminator(img)

In [ ]:
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
train(epochs=30000, batch_size=16, save_interval=1000)

1/1 [==============================] - 0s 169ms/step
0 [D loss: 0.951528, acc.: 43.75%] [G loss: 0.783294]
1/1 [==============================] - 0s 27ms/step
1 [D loss: 0.518696, acc.: 87.50%] [G loss: 0.659467]
1/1 [==============================] - 0s 30ms/step
2 [D loss: 0.637408, acc.: 68.75%] [G loss: 0.541458]
1/1 [==============================] - 0s 36ms/step
3 [D loss: 0.572582, acc.: 50.00%] [G loss: 0.516226]
1/1 [==============================] - 0s 28ms/step
4 [D loss: 0.685313, acc.: 56.25%] [G loss: 0.251082]
1/1 [==============================] - 0s 30ms/step
5 [D loss: 0.864772, acc.: 56.25%] [G loss: 0.494453]
1/1 [==============================] - 0s 28ms/step
6 [D loss: 0.988689, acc.: 56.25%] [G loss: 0.232225]
1/1 [==============================] - 0s 83ms/step
7 [D loss: 1.180232, acc.: 50.00%] [G loss: 0.701150]
1/1 [==============================] - 0s 19ms/step
8 [D loss: 1.155369, acc.: 56.25%] [G loss: 0.837875]
1/1 [==============================] - 0s 19m

In [ ]:
generator.save('hydra_generator_30kepochs.h5')

In [ ]:
# r, c = 5, 5
# noise = np.random.normal(0, 1, (r * c, 100))
# gen_imgs = generator.predict(noise)

# # Rescale images 0 - 1
# gen_imgs = 0.5 * gen_imgs + 0.5

# # Create the 'images_do/' directory if it doesn't exist
# os.makedirs('images_dot/', exist_ok=True)

# fig, axs = plt.subplots(r, c)
# cnt = 0
# for i in range(r):
#     for j in range(c):
#         axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
#         # axs[i, j].imshow(gen_imgs[cnt, :, :, :])
#         axs[i, j].axis('off')
#         cnt += 1

# fig.savefig("images_dot/mnist_%d.png")
# plt.close()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')